In [2]:
%reset -f

import numpy as np
import numba as  nb
import pandas as pd

all = pd.read_parquet('tmp.parq')
subset = all[-10000:]

In [3]:

@nb.jit(nopython=True)
def do_a_manual(t:np.array, val:np.array, bin_start:np.array, ) -> np.array:
    n_ticks, n_bins = len(t), len(np.unique(bin_start))
    i, j, ii = 0, 0, 0
    bin_index = np.full(n_bins, 0)
    n_samples = np.full(n_bins, 0)
    o = np.full(n_bins, 0.0)
    h = np.full(n_bins, 0.0)
    l = np.full(n_bins, 0.0)
    c = np.full(n_bins, 0.0)

    while j < n_ticks:
        while (j < n_ticks) & (bin_start[i] == bin_start[j]):
            j += 1
        # i is the index of the first element on the binned timestamp
        # je is the index to the last element of the binned timestamp
        # ii is the insertion point for the current bin

        bin_index[ii] = bin_start[j - 1]
        n_samples[ii] = j - i

        bin_times = t[i:j]
        bin_ticks = val[i:j]

        o[ii] = bin_ticks[0]
        h[ii] = np.max(bin_ticks)
        l[ii] = np.min(bin_ticks)
        c[ii] = bin_ticks[-1]

        # np.append(np.array([1,2,3]),1)
        if ii > 0:
            bb =  -1 #insert_int(bin_times, 0, t[i])
            # bb = np.append(t[i], bin_times.copy())
            # bin_ticks = np.append(c[ii-1], bin_ticks)


        ii += 1
        i = j
            
    return bin_index, n_samples, o, h, l, c

#pad_on_bin_bopundary
def foo(ticks:pd.DataFrame,
        bin_size_sec:int,
        time_col:str = "t",
        ) -> pd.DataFrame:

    bin_start = ticks[time_col].values.astype(f'datetime64[{bin_size_sec}s]').astype('datetime64[ms]').astype(np.int64)
    mid = (ticks.bid.values + ticks.ask.values) / 2
    bin_index, n_samples, o, h, l, c = do_a_manual(ticks.t.values, mid, bin_start)
    return pd.DataFrame({'bin_start':bin_index.astype('datetime64[ms]'), 'n_samples':n_samples, 'o':o, 'h':h, 'l':l, 'c':c})

res = foo(subset, 60)
res[:5]

,bin_start,n_samples,o,h,l,c
0,2023-06-22 08:24:00,31,1.162395,1.162455,1.162355,1.162355
1,2023-06-22 08:25:00,34,1.162345,1.162345,1.162115,1.162185
2,2023-06-22 08:26:00,30,1.162175,1.162185,1.162005,1.162065
3,2023-06-22 08:27:00,40,1.162005,1.162015,1.161865,1.161865
4,2023-06-22 08:28:00,32,1.161875,1.161915,1.161775,1.161915


In [4]:
res = foo(all, 60)
res

,bin_start,n_samples,o,h,l,c
0,2023-06-06 19:22:00,18,1.162185,1.162205,1.162185,1.162195
1,2023-06-06 19:23:00,7,1.162195,1.162205,1.162165,1.162165
2,2023-06-06 19:27:00,4,1.162165,1.162175,1.162165,1.162165
3,2023-06-06 19:29:00,3,1.162165,1.162185,1.162165,1.162185
4,2023-06-06 20:27:00,6,1.161995,1.162005,1.161995,1.161995
...,...,...,...,...,...,...
9952,2023-06-22 12:33:00,52,1.161965,1.162075,1.161835,1.161885
9953,2023-06-22 12:34:00,49,1.161905,1.162255,1.161885,1.162255
9954,2023-06-22 12:35:00,49,1.162285,1.162335,1.162035,1.162095
9955,2023-06-22 12:36:00,52,1.162045,1.162055,1.161845,1.161895


In [59]:
@nb.jit(nopython=True)
def insert_int(a:np.array, idx, v:np.int64) -> np.array:
    N = len(a)
    M = N + 1

    if idx < 0:
        if np.abs(idx) > M:
            raise Exception(f"Insertion index out of range {idx}.")
    else:
        if np.abs(idx) > N:
            raise Exception(f"Insertion index out of range {idx}.")

    # standard python behaviour. allows insertions from rear    
    insertion_pt = idx % M

    tgt = np.full(M, 0)
        
    for i in range (0, insertion_pt):
        tgt[i] = a[i]
    tgt[insertion_pt] = v
    for i in range (insertion_pt, N):
        tgt[i+1] = a[i]

    return tgt


In [72]:
a = np.array([1,2,3,4])
b = insert_int(a, 4, -4)
b


array([ 1,  2,  3,  4, -4])

In [58]:
[0,1,2,3][-4]

0